In [6]:
from glob import glob
import pandas as pd
import numpy as np
import csv
import os
%pylab inline
import matplotlib.pyplot as plt
from subprocess import check_output
from subprocess import Popen, PIPE
import shutil
from nipype.interfaces import fsl
from nipype.interfaces.fsl import RobustFOV, Reorient2Std
from nipype.interfaces.c3 import C3dAffineTool
import json
from pprint import pprint

Populating the interactive namespace from numpy and matplotlib


In [ ]:
function to find what TP current mse is (hopefully some master list exists that matches mseid to msid + TP)
if TP = 1, do nothing
if TP != 1, then:
    find TP1’s affine.txt
        if none, either run TP1’s alignment, or raise error that TP1 alignment not run yet
    convert TP1’s affine.txt and currentTP’s affine.txt to concat.mat (convert_xfm)
    T2/lesion mask/other images + concat.mat (in)
    T1MNI.nii.gz (ref)
    T2/FLAIR/lesion mask/other images in T1_baseline_MNI space

In [4]:
TP1_base_dir = "/data/henry7/PBR/subjects/mse1872/alignment/"

In [15]:
TP2_base_dir = "/data/henry7/PBR/subjects/mse3644/alignment/"

In [71]:
with open(TP1_base_dir + "/status.json") as data_file:
    data = json.load(data_file)
    t1_tp1 = data["t1_files"][-1]
    t2_tp1 = data["t2_files"][-1]
    #gad_tp1 = data["gad_files"][-1]
    #flair_tp1 = data["flair_files"][-1]
    affines_tp1 = data["affines"]
    T1_BL_MNI = TP1_base_dir +"mni_angulated/"+os.path.split(t1_tp1)[-1].replace(".nii.gz", "_trans.nii.gz")
    print(t1, t2, affines_tp1, T1_BL_MNI)

/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-002-AX_T1_3D_IRSPGR.nii.gz /data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001.nii.gz ['/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-000_affine.txt', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine.txt', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-009-AX_T1_affine.txt'] /data/henry7/PBR/subjects/mse1872/alignment/mni_angulated/ms1060-mse1872-002-AX_T1_3D_IRSPGR_trans.nii.gz


In [57]:
with open(TP2_base_dir + "/status.json") as data_file:
    data = json.load(data_file)
    t1_tp2 = data["t1_files"][-1]
    t2_tp2 = data["t2_files"][-1]
    affines_tp2 =data["affines"]
    print(t1, t2, affines)

/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-002-AX_T1_3D_IRSPGR.nii.gz /data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001.nii.gz ['/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-000_affine.txt', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine.txt', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-009-AX_T1_affine.txt']


In [58]:
mni_template = "/data/henry6/PBR/templates/OASIS-30_Atropos_template_in_MNI152.nii.gz"

# if TP == 1:

In [78]:
#if align exists
#convert t1 mni matrix to .mat 
cmd = ["c3d_affine_tool", "-itk",TP1_base_dir + "/mni_angulated/affine.txt", "-o",TP1_base_dir + "/mni_angulated/affine.mat"]
proc = Popen(cmd, stdout=PIPE)
#convert other affines in align folder to .mat 
for affine in affines:
    print(affine)
    cmd = ["c3d_affine_tool", "-itk", affine, "-o",affine.split(".")[0]+ ".mat"]
    proc = Popen(cmd, stdout=PIPE)
    print(cmd)

/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-000_affine.txt
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-000_affine.txt', '-o', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-000_affine.mat']
/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine.txt
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine.txt', '-o', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine.mat']
/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-009-AX_T1_affine.txt
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-009-AX_T1_affine.txt', '-o', '/data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-009-AX_T1_affine.mat']


In [62]:
#combine affine matrices 
for affine in affines:
    invt = fsl.ConvertXFM()
    invt.inputs.in_file = affine.split(".")[0]+ ".mat"
    invt.inputs.in_file2 = TP1_base_dir + "/mni_angulated/affine.mat"
    invt.inputs.concat_xfm = True
    invt.inputs.out_file = affine.split(".")[0] + "_mni.mat"
    invt.cmdline
    invt.run()

In [64]:
#apply affine matrices... register the T2's 
flt = fsl.FLIRT()
flt.inputs.cost = "mutualinfo"
flt.inputs.in_file = t2_tp1 
flt.inputs.reference = T1_BL_MNI = TP1_base_dir +"mni_angulated/" +os.path.split(t1_tp1)[-1].replace(".nii.gz", "_trans.nii.gz")
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.in_matrix_file = t2_tp1.replace(".nii.gz", "_affine_mni.mat")
flt.inputs.out_file = t2_tp1.replace(".nii.gz", "_T1mni.nii.gz")
print(flt.cmdline)
flt.run() 
              

flirt -in /data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001.nii.gz -ref /data/henry7/PBR/subjects/mse1872/alignment/mni_angulated/ms1060-mse1872-002-AX_T1_3D_IRSPGR_trans.nii.gz -out /data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_T1mni.nii.gz -omat ms1060-mse1872-003-FRFSE-001_flirt.mat -cost mutualinfo -init /data/henry7/PBR/subjects/mse1872/alignment/ms1060-mse1872-003-FRFSE-001_affine_mni.mat


In [72]:
#else --> run pbr align 

# if TP != 1 

In [79]:
#if align exists
#convert all affines in align folder to .mat 
for affine2 in affines_tp2:
    cmd = ["c3d_affine_tool", "-itk", affine2, "-o",affine2.split(".")[0]+ ".mat"]
    proc = Popen(cmd, stdout=PIPE)
    print(cmd)

['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-003-AX_T1_3D_IRSPGR-001_affine.txt', '-o', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-003-AX_T1_3D_IRSPGR-001_affine.mat']
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-000_affine.txt', '-o', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-000_affine.mat']
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-001_affine.txt', '-o', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-001_affine.mat']
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-002_affine.txt', '-o', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-002_affine.mat']
['c3d_affine_tool', '-itk', '/data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-003_affine.txt', '-o', '/data/henry7/PBR/subjects/

In [80]:
#combine mni t1 affine with the align affines 
for affine2 in affines_tp2:
    invt = fsl.ConvertXFM()
    invt.inputs.in_file = affine2.split(".")[0]+ ".mat"
    invt.inputs.in_file2 = TP1_base_dir + "/mni_angulated/affine.mat"
    invt.inputs.concat_xfm = True
    invt.inputs.out_file = affine2.split(".")[0] + "_mni.mat"
    invt.cmdline
    invt.run()

In [81]:
#apply affine matrices... register the T1's 
flt = fsl.FLIRT()
flt.inputs.cost = "mutualinfo"
flt.inputs.in_file = t1_tp2
flt.inputs.reference = T1_BL_MNI 
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.in_matrix_file = t1_tp2.replace(".nii.gz", "_affine_mni.mat")
flt.inputs.out_file = t1_tp2.replace(".nii.gz", "_T1mni.nii.gz")
print(flt.cmdline)
flt.run() 

flirt -in /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-003-AX_T1_3D_IRSPGR-001.nii.gz -ref /data/henry7/PBR/subjects/mse1872/alignment/mni_angulated/ms1060-mse1872-002-AX_T1_3D_IRSPGR_trans.nii.gz -out /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-003-AX_T1_3D_IRSPGR-001_T1mni.nii.gz -omat ms1060-mse3644-003-AX_T1_3D_IRSPGR-001_flirt.mat -cost mutualinfo -init /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-003-AX_T1_3D_IRSPGR-001_affine_mni.mat


In [82]:
#apply affine matrices... register the T2's 
flt = fsl.FLIRT()
flt.inputs.cost = "mutualinfo"
flt.inputs.in_file = t2_tp2
flt.inputs.reference = T1_BL_MNI 
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.in_matrix_file = t2_tp2.replace(".nii.gz", "_affine_mni.mat")
flt.inputs.out_file = t2_tp2.replace(".nii.gz", "_T1mni.nii.gz")
print(flt.cmdline)
flt.run()

flirt -in /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-003.nii.gz -ref /data/henry7/PBR/subjects/mse1872/alignment/mni_angulated/ms1060-mse1872-002-AX_T1_3D_IRSPGR_trans.nii.gz -out /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-003_T1mni.nii.gz -omat ms1060-mse3644-005-FRFSE-003_flirt.mat -cost mutualinfo -init /data/henry7/PBR/subjects/mse3644/alignment/ms1060-mse3644-005-FRFSE-003_affine_mni.mat
